### Interest Over Interest on Bond

In [3]:
import pandas as pd
from absbox.local.generic import Generic
pd.set_option("display.max_rows",None)

#### Interest Due

In [130]:
interestDueExample = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-04-01","firstPay":"2021-06-20"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthFirst","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":20
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),("acc02",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.30
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,tuple()
    ,{"amortizing":[
         ["accrueAndPayInt","acc02",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc02"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [133]:
from absbox import API,EnginePath
localAPI = API(EnginePath.DEV,lang='english',check=False)
#localAPI = API("https://absbox.org/api/dev",lang='english',check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.26.4, server:0.27.21

In [134]:
r = localAPI.run(interestDueExample
               ,poolAssump = None
               ,runAssump = None
               ,read=True)

Warning Message from server:
Account acc02 has cash to be distributed

In [136]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2021-06-20,583.05,54.49,416.95,0.3,471.44,11.26,0,0.58305,"[<PayInt:A1>, <PayPrin:A1>]"
2021-07-20,477.08,11.88,105.97,0.3,117.85,13.75,0,0.47708,"[<PayInt:A1>, <PayPrin:A1>]"
2021-08-20,370.39,11.17,106.69,0.3,117.86,14.73,0,0.37039,"[<PayInt:A1>, <PayPrin:A1>]"
2021-09-20,263.00,10.46,107.39,0.3,117.85,13.70,0,0.26300,"[<PayInt:A1>, <PayPrin:A1>]"
2021-10-20,154.90,9.75,108.10,0.3,117.85,10.43,0,0.15490,"[<PayInt:A1>, <PayPrin:A1>]"
2021-11-20,46.07,9.03,108.83,0.3,117.86,5.34,0,0.04607,"[<PayInt:A1>, <PayPrin:A1>]"
2021-12-20,0.00,6.47,46.07,0.3,52.54,0.00,0,0.00000,"[<PayInt:A1>, <PayPrin:A1>]"


##### Backout Date: 2021-07-20

In [137]:
583.05 * 0.3 * 30/365 # new interest accrued

14.376575342465753

In [138]:
14.376575342465753 + 11.26 # remain interest due

25.636575342465754

In [139]:
25.636575342465754 - 11.88 ## remaining interest due at end of 2021-07-20

13.756575342465753

##### Backout Date: 2021-12-20

In [140]:
# total due int at begin of 2021-12-20

46.07 * 0.3 * 30/365 + 5.34

6.475972602739726

In [141]:
# remain interest due at end of 2021-12-20
6.475972602739726 - 6.47

0.00597260273972644

#### Interest Over Interest

In [142]:
from lenses import lens

interestDueExample.bonds

(('A1',
  {'balance': 1000,
   'rate': 0.3,
   'originBalance': 1000,
   'originRate': 0.07,
   'startDate': '2020-01-03',
   'rateType': {'Fixed': 0.08},
   'bondType': {'Sequential': None}}),
 ('B',
  {'balance': 1000,
   'rate': 0.0,
   'originBalance': 1000,
   'originRate': 0.07,
   'startDate': '2020-01-03',
   'rateType': {'Fixed': 0.0},
   'bondType': {'Equity': None}}))

In [143]:
interestOverInterestType = ("withIntOverInt", ("inflate", 0.2), {"fix":0.001} )

interestOverInterestExample = lens.bonds[0][1]['rateType'].set(interestOverInterestType)(interestDueExample)

In [144]:
interestOverInterestExample.bonds[0]

('A1',
 {'balance': 1000,
  'rate': 0.3,
  'originBalance': 1000,
  'originRate': 0.07,
  'startDate': '2020-01-03',
  'rateType': ('withIntOverInt', ('inflate', 0.2), {'fix': 0.001}),
  'bondType': {'Sequential': None}})

In [145]:
r = localAPI.run(interestOverInterestExample
               ,poolAssump = None
               ,runAssump = None
               ,read=True)

Warning Message from server:
Account acc02 has cash to be distributed

In [146]:
r['bonds']['A1'].loc["2021-07-20":"2021-08-20"]

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2021-07-20,477.08,11.88,105.97,0.3,117.85,14.08,0,0.47708,"[<PayInt:A1>, <PayPrin:A1>]"
2021-08-20,370.39,11.17,106.69,0.3,117.86,15.49,0,0.37039,"[<PayInt:A1>, <PayPrin:A1>]"


##### Backout 2021-08-20

In [147]:
# new due int amount
477.08 * 0.3 * 31/365

12.15573698630137

In [148]:
# total due int at begin of 2021-08-20
12.15573698630137 + 14.08

26.23573698630137

In [149]:
# int over int
14.08 * (0.3*1.2) * 31/365

0.43050082191780814

In [150]:
# amount to paid of interest due
(11.17 - 0.43050082191780814)

10.739499178082191

In [151]:
## due int after interest payment
26.23573698630137 - 10.739499178082191

15.496237808219178